# Préparation et modélisation des données

In [1]:
# Import package
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

# Chargement des données
bc_data = pd.read_csv('breast-cancer-data.csv', index_col=0)


In [2]:
# Information de DF
bc_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 569 entries, 842302 to 92751
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                 

Aucune variable ne contient de valeurs manquantes.
Toutes les variables, à l'exception de 'diagnosis' sont des variables continues, mais les arbres de décisions fonctionnent aussi bien avec des variables discrètes ou qualitatives (après dichotomisation sur scikit-learn).
Un coup d'oeil aux premières lignes du Data Frame nous montre que les différentes variables ne sont pas à la même échelle, mais très peu de "preprocessing" est nécessaire lors d'une classification par arbre de décision.

In [3]:
# Création DF dta contenant les features
data = bc_data.drop('diagnosis', axis=1)

# Création vecteur target avec variable cible 'diagnosis'
target = bc_data.diagnosis

# Apprentissage des données

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data,target,test_size=0.2, random_state=123)

dt_clf = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=123)
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=123)

In [5]:
#Application du modèle
y_pred = dt_clf.predict(X_test)

#Matrice de confusion
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,B,M
Classe réelle,,
B,72,1
M,3,38


Les prédictions faites sur l’échantillon test sont encourageantes

In [7]:
# Affichage des 8 variable les plus importantes pour dt_clf, et leurs importances respectives
feats = {}
for feature, importance in zip(data.columns, dt_clf.feature_importances_):
    feats[feature] = importance 
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'})
importances.sort_values(by='Importance', ascending=False).head(8)


,Importance
radius_worst,0.620770
concave points_worst,0.177674
concavity_mean,0.060736
texture_mean,0.051408
texture_worst,0.041369
texture_se,0.020865
area_worst,0.016107
fractal_dimension_worst,0.011072


Utilisation de la mesure d'impureté : index Gini. 

In [8]:
dt_clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=321)

dt_clf_gini.fit(X_train, y_train)

y_pred = dt_clf_gini.predict(X_test)

pd.crosstab(y_test,y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,B,M
Classe réelle,,
B,72,1
M,2,39


In [9]:
feats = {}
for feature, importance in zip(data.columns, dt_clf_gini.feature_importances_):
    feats[feature] = importance 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})

#Affichage des 8 variables les plus importantes
importances.sort_values(by='Gini-importance', ascending=False).head(8)


,Gini-importance
radius_worst,0.718582
concave points_worst,0.112821
texture_worst,0.048798
concavity_mean,0.033920
texture_mean,0.030343
area_worst,0.016458
smoothness_mean,0.011231
area_se,0.009682
